model with 99% accuracy and 44kb size,  256 filters and alpha=.25 -----> 1671405237

model with 99% accuracy and 15kb size,  128 filters and alpha=.25 -----> 1671409506  

model with 98% accuracy and 44kb size,  128 filters and alpha=.5 -----> 1671410314

In [4]:
import tensorflow as tf
import os
import numpy as np
import random


In [5]:
go_stop_train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
go_stop_val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
go_stop_test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])

2022-12-19 00:24:22.849346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-19 00:24:22.849382: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-19 00:24:22.849414: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-1971cb16-e709-4b3f-8d6d-f6f260f3f6b7): /proc/driver/nvidia/version does not exist
2022-12-19 00:24:22.849778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
num_train_files = len(go_stop_train_ds)
num_val_files = len(go_stop_val_ds)
num_test_files = len(go_stop_test_ds)

print('Training set size:', num_train_files)
print('Validation set size:', num_val_files)
print('Test set size:', num_test_files)

Training set size: 1600
Validation set size: 200
Test set size: 200


In [45]:
TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 10
}
final_sparsity = 0.70
alpha =0.5

In [46]:
from preprocessing import LABELSB
from functools import partial
from collections import namedtuple
from itertools import product
import tensorflow_model_optimization as tfmot

parameters = dict(
    
    downsampling_rate = [16000],#[16000,32000], 
    frame_length_in_s = [0.016],     
    frame_step_in_s = [0.012],  
    num_mel_bin =[10],#[10, 40],
    lower_frequency =[20],#[20, 80],
    upper_frequency = [8000],#[2000, 8000],
    num_coefficients=[40],#[10,40]
    
)

class RunBuilder():
  @staticmethod
  def get_runs(params):
    Run = namedtuple('Run', params.keys())
    runs=[]
    for v in product(*params.values()):
       runs.append(Run(*v))
    return runs

def preprocess(filename):
        signal, downsmpling_rate,label = get_frozen_spectrogram(filename)
        signal.set_shape(SHAPE)
        signal = tf.expand_dims(signal, -1)
        signal = tf.image.resize(signal, [32, 32])
        label_id = tf.argmax(label == LABELSB)

        return signal, label_id


from preprocessing import get_spectrogram,get_log_mel_spectrogram,LABELSB
tmplist=[]
for runs in RunBuilder.get_runs(parameters):
    PREPROCESSING_ARGS = {
    'downsampling_rate': runs.downsampling_rate,
    'frame_length_in_s': runs.frame_length_in_s,
    'frame_step_in_s': runs.frame_step_in_s,
    'num_mel_bin':runs.num_mel_bin,
    'lower_frequency':runs.lower_frequency,
    'upper_frequency':runs.upper_frequency,
    'num_coefficients':runs.num_coefficients
    
    }

    # Use Fallowing code to train on spectrogram
    #get_frozen_spectrogram = partial(get_spectrogram, **PREPROCESSING_ARGS)
    #final_ds = go_stop_train_ds.map(get_frozen_spectrogram)    
    #for spectrogram, downsampling_rate, label in final_ds.take(1):
     #  SHAPE = spectrogram.shape

    
    # Use Following code to train on mfcc    
    get_frozen_spectrogram = partial(get_log_mel_spectrogram, **PREPROCESSING_ARGS)
    final_ds = go_stop_train_ds.map(get_frozen_spectrogram)
    for spectrogram, downsampling_rate, label in final_ds.take(1):
        SHAPE = spectrogram.shape
    batch_size = TRAINING_ARGS['batch_size']
    epochs = TRAINING_ARGS['epochs']

    train_ds = go_stop_train_ds.map(preprocess).batch(batch_size).cache()
    val_ds = go_stop_val_ds.map(preprocess).batch(batch_size)
    test_ds = go_stop_test_ds.map(preprocess).batch(batch_size)
   

    for example_batch, example_labels in train_ds.take(1):
        print()
        print('Batch Shape:', example_batch.shape)
        print('Data Shape:', example_batch.shape[1:])
        print('Labels:', example_labels)
   
    
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(128*alpha), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], 
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(128*alpha), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(128*alpha), kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELSB)),
    tf.keras.layers.Softmax()
])

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    begin_step = int(len(go_stop_train_ds) * epochs * 0.2)
    end_step = int(len(go_stop_train_ds) * epochs)

    pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # model.summary()
    

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
    initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
    end_learning_rate = TRAINING_ARGS['end_learning_rate']

    linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=initial_learning_rate,
        end_learning_rate=end_learning_rate,
        decay_steps=len(train_ds) * epochs,
    )
    optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
    metrics = [tf.metrics.SparseCategoricalAccuracy()]
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)
    test_loss, test_accuracy = model.evaluate(test_ds)
    training_loss = history.history['loss'][-1]
    training_accuracy = history.history['sparse_categorical_accuracy'][-1]
    val_loss = history.history['val_loss'][-1]
    val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

    print(f'Training Loss: {training_loss:.4f}')
    print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
    print()
    print(f'Validation Loss: {val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
    print()
    print(f'Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_accuracy*100.:.2f}%')
    record = {
    'downsampling_rate': runs.downsampling_rate,
    'frame_length_in_s': runs.frame_length_in_s,
    'frame_step_in_s': runs.frame_step_in_s,
    'num_mel_bin':runs.num_mel_bin,
    'lower_frequency':runs.lower_frequency,
    'upper_frequency':runs.upper_frequency,
    'num_coefficients':runs.num_coefficients,
    'Training Accuracy': training_accuracy*100,
    'Validation Accuracy': val_accuracy*100,
    'Test Accuracy':test_accuracy*100
    }
    tmplist.append(record)
    print(record)

2022-12-19 00:37:51.072397: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 00:37:51.074116: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 00:37:51.074312: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 00:37:51.715594: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 00:37:51.717556: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 00:37:51.717764: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-19 00:37:52.087664: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed 

In [47]:
from time import time

timestamp = int(time())

saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1671410314/assets
INFO:tensorflow:Assets written to: ./saved_models/1671410314/assets


In [48]:
import pandas as pd

output_dict = {
    'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])

output_path='./spectrogram_dscnn_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)


In [49]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{1671409506}')
tflite_model = converter.convert()

2022-12-19 00:38:54.152188: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-19 00:38:54.152243: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-19 00:38:54.152385: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/1671410314
2022-12-19 00:38:54.155238: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-19 00:38:54.155285: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/1671410314
2022-12-19 00:38:54.164475: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-19 00:38:54.233642: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./saved_models/1671410314
2022-12-19 00:38:54.248554: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status

In [50]:
tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

In [51]:
tflite_model_name1 = os.path.join(tflite_models_dir, f'{1671409506}.tflite')
tflite_model_name1

'./tflite_models/1671410314.tflite'

In [52]:
with open(tflite_model_name1, 'wb') as fp:
    fp.write(tflite_model)

In [53]:
import zipfile

with zipfile.ZipFile(f'{tflite_model_name1}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name1)

In [54]:
import zipfile

not_pruned_tflite = os.path.join(tflite_models_dir, '1671409506.tflite')

with zipfile.ZipFile(f'{not_pruned_tflite}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(not_pruned_tflite)  

In [ ]:
not_pruned_tflite_size = os.path.getsize(not_pruned_tflite) / 1024.0
tflite_size = os.path.getsize(tflite_model_name1) / 1024.0
not_pruned_zipped_size = os.path.getsize(f'{not_pruned_tflite}.zip') / 1024.0
zipped_size = os.path.getsize(f'{tflite_model_name1}.zip') / 1024.0

print(f'Original tflite size (not pruned model): {tflite_size:.3f} KB')
print(f'Original tflite size (pruned model): {tflite_size:.3f} KB')
print(f'Zipped tflite size (not pruned model): {not_pruned_zipped_size:.3f} KB')
print(f'Zipped tflite size (pruned model): {zipped_size:.3f} KB')

In [56]:
MODEL_NAME = tflite_model_name1
MODEL_NAME

'./tflite_models/1671410314.tflite'

In [57]:
downsampling_rate = PREPROCESSING_ARGS['downsampling_rate']
sampling_rate_int64 = tf.cast(downsampling_rate, tf.int64)
frame_length = int(downsampling_rate * PREPROCESSING_ARGS['frame_length_in_s'])
frame_step = int(downsampling_rate * PREPROCESSING_ARGS['frame_step_in_s'])
spectrogram_width = (16000 - frame_length) // frame_step + 1
num_spectrogram_bins = frame_length // 2 + 1

linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    PREPROCESSING_ARGS['num_mel_bin'],
    num_spectrogram_bins,
    downsampling_rate,
    PREPROCESSING_ARGS['lower_frequency'],
    PREPROCESSING_ARGS['upper_frequency']
)

In [84]:
interpreter1 = tf.lite.Interpreter(model_path='tflite_models/1671409506.tflite')
interpreter1.allocate_tensors()

input_details = interpreter1.get_input_details()
output_details = interpreter1.get_output_details()

print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Input index:", input_details[0]['index'])
print("Output name:", output_details[0]['name'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_input_2:0
Input shape: [ 1 32 32  1]
Input index: 0
Output name: StatefulPartitionedCall:0
Output shape: [1 2]


In [85]:
downsampling_rate= 16000
frame_length_in_s= 0.016
frame_step_in_s= 0.012
num_mel_bins= 10
lower_frequency= 20
upper_frequency= 8000
num_coefficients=40

In [86]:
sampling_rate_int64 = tf.cast(downsampling_rate, tf.int64)
frame_length = int(downsampling_rate * frame_length_in_s)
frame_step = int(downsampling_rate * frame_step_in_s)
spectrogram_width = (16000 - frame_length) // frame_step + 1
num_spectrogram_bins = frame_length // 2 + 1

linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    num_mel_bins,
    num_spectrogram_bins,
    downsampling_rate,
    lower_frequency,
    upper_frequency
)

In [87]:
print(interpreter1.get_input_details())

[{'name': 'serving_default_input_2:0', 'index': 0, 'shape': array([ 1, 32, 32,  1], dtype=int32), 'shape_signature': array([-1, 32, 32,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [88]:
"""
Inference 
"""
from glob import glob
from time import time
import tensorflow_io as tfio


#filenames = glob('msc-test/*')
filenames = glob('msc-test/go*') + glob('msc-test/stop*')


avg_preprocessing_latency = 0.0
avg_model_latency = 0.0
avg_total_latency = 0.0
accuracy = 0.0

for filename in filenames:
    audio_binary = tf.io.read_file(filename)
    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    true_label = file_parts[0]
    true_label = true_label.numpy().decode()
    
    start_preprocess = time()
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 
    audio = tf.squeeze(audio)
    zero_padding = tf.zeros(sampling_rate - tf.shape(audio), dtype=tf.float32)
    audio_padded = tf.concat([audio, zero_padding], axis=0)

    if downsampling_rate != sampling_rate:
        audio_padded = tfio.audio.resample(audio_padded, sampling_rate_int64, downsampling_rate)

    stft = tf.signal.stft(
        audio_padded, 
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    spectrogram = tf.abs(stft)

    mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)[..., :num_coefficients]

    
    SHAPE=mfccs.shape
    mfccs.set_shape(SHAPE)
    mfccs = tf.expand_dims(mfccs, 0)
    mfccs = tf.expand_dims(mfccs, -1)
    
    mfccs = tf.image.resize(mfccs, [32, 32])
   
    label_id = tf.argmax(label == LABELSB)
    
    end_preprocess = time()

    interpreter1.set_tensor(input_details[0]['index'], mfccs)

    interpreter1.invoke()
    output = interpreter1.get_tensor(output_details[0]['index'])

    end_inference = time()

    top_index = np.argmax(output[0])
    predicted_label = LABELSB[top_index]

    accuracy += true_label == predicted_label
    avg_preprocessing_latency += end_preprocess - start_preprocess
    avg_model_latency += end_inference - end_preprocess
    avg_total_latency += end_inference - start_preprocess

In [89]:
print(accuracy)
accuracy /= len(filenames)
avg_preprocessing_latency /= len(filenames)
avg_model_latency /= len(filenames)
avg_total_latency /= len(filenames)

196.0


In [90]:
print(f'Accuracy: {100 * accuracy:.3f}%')
# print(f'Model size: {model_size / 2 ** 10:.1f}KB')
print(f'Preprocessing Latency: {1000 * avg_preprocessing_latency:.1f}ms')
print(f'Model Latency: {1000 * avg_model_latency:.1f}ms')
print(f'Total Latency: {1000 * avg_total_latency:.1f}ms')

Accuracy: 98.000%
Preprocessing Latency: 6.3ms
Model Latency: 0.2ms
Total Latency: 6.5ms


In [ ]:
import os
model_size = os.path.getsize(f'tflite_models/1671409506.tflite')/1024
print(model_size)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1971cb16-e709-4b3f-8d6d-f6f260f3f6b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>